In [1]:
import gensim
import smart_open
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        loop = 0
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

            loop += 1
            if loop % 10000 == 0:
                print(loop)

#             if loop == 1000000:
#                 break

In [4]:
documents = list(read_corpus('./data/dataTitle.txt'))

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000


In [5]:
len(documents)

1000000

In [6]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0
        self.batch = 1

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

    def on_train_begin(self, model):
        print("train started")
        self.batch = 1

    def on_train_end(self, model):
        print("train ended")

In [7]:
models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=2, epochs=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=2, epochs =10, workers=cores),
]

In [8]:
models[0].build_vocab(documents)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,mc2,s0.001,t8)
Doc2Vec(dm/m,d200,n5,w8,mc2,s0.001,t8)


In [9]:
for index, model in enumerate(models):
    %time model.train(documents, total_examples=model.corpus_count, epochs=self.epochs, callbacks=[EpochLogger()])
    model.save('./test-model/data' + str(index) + ".model")

train started
Epoch #0 start


/home/iamdipta/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
train ended
CPU times: user 51min 30s, sys: 9min 22s, total: 1h 52s
Wall time: 21min 2s
train started
Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
train ended
CPU times: user 32min 35s, sys: 14min 52s, total: 47min 27s
Wall time: 24min 40s


In [16]:
for index, model in enumerate(models):
    models[index] = Doc2Vec.load('./model/data' + str(index) + ".model")

In [17]:
from pprint import pprint
def get(model, st):
    tokens = st.lower().split()
    new_vector = model.infer_vector(tokens)
    sims = model.docvecs.most_similar([new_vector])
    for index, val in sims:
        print(str(val) + " " + str(' '.join(documents[index].words)))

In [18]:
guess_it = "macbook pro retina" 
for model in models:
    print(str(model))
    get(model, guess_it)
    print("\n\n")
    
# Have to use dbow

Doc2Vec(dbow+w,d200,n5,w8,mc2,s0.001,t8)
0.8697952032089233 neoprene pro carrying case sleeve for macbook air macbook pro macbook pro retina display black
0.8628667593002319 uniharpa inch laptop sleeve pu case cover for macbook air retina macbook pro inch ipad pro black
0.8584039807319641 us keyboard for macbook retina
0.852268397808075 thule subterra macbook pro sleeve inch gray
0.849841296672821 felt laptop sleeve for inch macbook air retina macbook pro inch ipad blue
0.8488801717758179 thule gauntlet sleeve for macbook pro macbook pro with retina display blacks
0.8471614122390747 mobility islice pro for macbook pro retina mid early red
0.8420846462249756 tomtoc protective laptop sleeve bag for inch macbook air macbook pro retina surface book surface laptop ipad pro chromebook tablet red
0.8413190841674805 tucano doppio backpack for macbook pro retina notebook black
0.841266393661499 kengel plastic hard case cover for macbook pro retina inch and black



Doc2Vec(dm/m,d200,n5,w8,mc2,s

In [26]:
models[0].docvecs.save_word2vec_format('./dbow.txt', binary=False)

In [40]:
EMBEDDING_FILE = "./data/dbow.txt"
DIMENSION_FILE = "./data/dataDimesions.txt"

In [38]:
X = []
Y = []

In [39]:
X = []
with open(EMBEDDING_FILE, "r") as ip:
    flg = 0
    
    for line in ip:
        if flg:
            data = line.split()
            del data[0]
            data = list(map(float, data))
            X.append(data)
        flg = 1

In [53]:
Y = []
with open(DIMENSION_FILE, "r") as op:
    for line in op:
        data = line.split()
        data = list(map(float, data))
        Y.append(data)
        if len(Y) == len(X):
            break

In [54]:
print(len(X), len(Y))
print(len(X[0]), len(Y[0]))

1000000 1000000
200 4


In [55]:
import numpy as np

docs = np.array(X)
target = np.array(Y)

ratio = int( docs.shape[0]/10 )
X_train = docs[ratio:,:]
X_test = docs[:ratio,:]
y_train = target[ratio:,:]
y_test = target[:ratio,:]

In [59]:
doc_embedding_size = 200
generations = 44

# Neural Network


In [ ]:
from tensorflow import keras

model = keras.Sequential()
activation_function = keras.layers.PReLU()

# Input Layer
model.add(keras.layers.Dense(405, kernel_initializer=keras.initializers.random_uniform, input_dim=doc_embedding_size,
                             kernel_regularizer=keras.regularizers.l2(0.01),
                             bias_regularizer=keras.regularizers.l2(0.01), name="input"))
model.add(activation_function)

# Hidden Layer
model.add(keras.layers.Dense(405, kernel_initializer='normal', name="hidden1"))
model.add(activation_function)
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(405, kernel_initializer='normal', name="hidden2"))
model.add(activation_function)
model.add(keras.layers.Dropout(0.2))

# Output Layer
model.add(keras.layers.Dense(4, kernel_initializer='normal', name="output", activation="relu"))

model.compile(loss='mean_absolute_error', optimizer="Adam", metrics=['mean_absolute_error'])

checkpoint = keras.callbacks.ModelCheckpoint("./model/best.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                                 mode='auto')
callbacks_list = []

# model = create_neural_model()
model.fit(X_train, y_train, epochs=generations, batch_size=256, validation_split=0.1, callbacks=callbacks_list)
print("Current one: ", model.evaluate(X_test, y_test, batch_size=256))

Train on 810000 samples, validate on 90000 samples
Epoch 1/44
810000/810000 [==============================] - 36s 44us/step - loss: 310.0006 - mean_absolute_error: 309.7186 - val_loss: 311.6571 - val_mean_absolute_error: 311.4698
Epoch 2/44
810000/810000 [==============================] - 38s 47us/step - loss: 306.1872 - mean_absolute_error: 305.9911 - val_loss: 311.0696 - val_mean_absolute_error: 310.8647
Epoch 3/44
810000/810000 [==============================] - 40s 49us/step - loss: 306.1307 - mean_absolute_error: 305.9253 - val_loss: 311.2237 - val_mean_absolute_error: 311.0166
Epoch 4/44
810000/810000 [==============================] - 37s 45us/step - loss: 306.0046 - mean_absolute_error: 305.8041 - val_loss: 310.2569 - val_mean_absolute_error: 310.0657
Epoch 5/44
810000/810000 [==============================] - 36s 44us/step - loss: 306.0001 - mean_absolute_error: 305.8159 - val_loss: 310.1602 - val_mean_absolute_error: 309.9906
Epoch 6/44
810000/810000 [=======================